In [1]:
# Load modules necessary
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
import pandas as pd
from scipy.signal import detrend

%matplotlib qt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Load sample dataset

##### For EEG dataset, we will use CHB-MIT Scalp EEG Dataset from here: https://physionet.org/content/chbmit/1.0.0/chb01/#files-panel

##### The original dataset contains 22 subjects. However, here in this tutorial we will explore data from Subject 01
##### You can download the dataset from Brightspace

# Vectors (norm, dot product, cosine rule)

In [2]:
# A vector is a quantity that has both magnitude and direction.
# The magnitude of a vector is a scalar, which is a quantity that has only magnitude.
# The dot product of two vectors is a scalar.

# The norm of a vector is a scalar that gives the length of the vector.
# The norm of a vector is denoted as ||v||, where v is the vector.
# For any vector v = [v1, v2, v3, ..., vn], the norm of the vector is given by:
# ||v|| = sqrt(v1^2 + v2^2 + v3^2 + ... + vn^2)
v = np.array([3, 4, 5])
norm_v = np.sqrt(np.sum(v**2))
print(f"The norm of the vector v = {v} is {norm_v}")

# The dot product of two vectors is a scalar that gives the product of the magnitudes of the two vectors and the cosine of the angle between them.
# The dot product of two vectors u = [u1, u2, u3, ..., un] and v = [v1, v2, v3, ..., vn] is given by:
# u.v = u1*v1 + u2*v2 + u3*v3 + ... + un*vn
u = np.array([1, 2, 3])
v = np.array([4, 5, 6])
dot_product = np.sum(u*v)
print(f"The dot product of the vectors u = {u} and v = {v} is {dot_product}")

# The cosine rule is a formula that relates the sides of a triangle to the cosine of one of its angles.
# The cosine rule is given by:
# u.v = ||u||*||v||*cos(theta)
# where theta is the angle between the vectors u and v.
# Rearranging the formula gives:
# cos(theta) = (u.v)/(||u||*||v||)
# The angle between two vectors can be calculated as:
# theta = arccos((u.v)/(||u||*||v||))
u = np.array([1, 2, 3])
v = np.array([4, 5, 6])
norm_u = np.sqrt(np.sum(u**2))
norm_v = np.sqrt(np.sum(v**2))
cos_theta = np.arccos(np.sum(u*v)/(norm_u*norm_v))
theta = np.degrees(cos_theta)
print(f"The angle between the vectors u = {u} and v = {v} is {theta} degrees")


The norm of the vector v = [3 4 5] is 7.0710678118654755
The dot product of the vectors u = [1 2 3] and v = [4 5 6] is 32
The angle between the vectors u = [1 2 3] and v = [4 5 6] is 12.933154491899135 degrees


# Filters as matrix multiplications

In [3]:
# Generate simulated EEG data
# The data is simulated as a 2D matrix with 5 channels and 1000 time points
n_channels = 5
n_times = 1000
data = np.empty((n_channels, n_times))
for i in range(n_channels):
    data[i, :] = np.cos(2*np.pi*10*np.linspace(0, 1, n_times) + i*np.pi/2) + np.random.randn(n_times)

# Plot the simulated EEG data
ncols = 3
nrows = int(np.ceil(n_channels/ncols))
f, axs = plt.subplots(nrows, ncols, figsize=(15, 10))
axs = axs.flatten()
for i in range(n_channels):
    ax = axs.flatten()[i]
    ax.plot(data[i, :])
    ax.set_title(f"Channel {i+1}")
    ax.set_xlabel("Time")
    ax.set_ylabel("Amplitude")

In [4]:
# Average Re-referencing via matrix multiplication
MeanMultiplier = np.eye(n_channels) - np.ones((n_channels, n_channels))/n_channels
print(MeanMultiplier)
data_re_ref = MeanMultiplier @ data

# # Plot the re-referenced EEG data
# f, axs = plt.subplots(nrows, ncols, figsize=(15, 10))
# axs = axs.flatten()
# for i in range(n_channels):
#     ax = axs.flatten()[i]
#     ax.plot(data_re_ref[i, :])
#     ax.set_title(f"Channel {i+1}")
#     ax.set_xlabel("Time")
#     ax.set_ylabel("Amplitude")

[[ 0.8 -0.2 -0.2 -0.2 -0.2]
 [-0.2  0.8 -0.2 -0.2 -0.2]
 [-0.2 -0.2  0.8 -0.2 -0.2]
 [-0.2 -0.2 -0.2  0.8 -0.2]
 [-0.2 -0.2 -0.2 -0.2  0.8]]


In [5]:
# Re-referencing data to a single channel using matrix multiplication
# Let's use channel 5 as the reference channel
ref_channel = 4
ReferenceMultiplier = np.eye(n_channels)
ReferenceMultiplier[:, ref_channel] = -1
ReferenceMultiplier[ref_channel, :] = 0
# ReferenceMultiplier[ref_channel, ref_channel] = 1
print(ReferenceMultiplier)
data_single_ref = ReferenceMultiplier @ data

# # Plot the single re-referenced EEG data
# f, axs = plt.subplots(nrows, ncols, figsize=(15, 10))
# axs = axs.flatten()
# for i in range(n_channels):
#     ax = axs.flatten()[i]
#     ax.plot(data_single_ref[i, :])
#     ax.set_title(f"Channel {i+1}")
#     ax.set_xlabel("Time")
#     ax.set_ylabel("Amplitude")

[[ 1.  0.  0.  0. -1.]
 [ 0.  1.  0.  0. -1.]
 [ 0.  0.  1.  0. -1.]
 [ 0.  0.  0.  1. -1.]
 [ 0.  0.  0.  0.  0.]]


In [6]:
# Electrode selection via matrix multiplication
# Let's select channels 1, 3, and 5
selected_channels = [0, 2, 4]
ChannelMultiplier = np.zeros((n_channels, n_channels))
for i in selected_channels:
    ChannelMultiplier[i, i] = 1
print(ChannelMultiplier)

data_selected = ChannelMultiplier @ data

[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]


In [7]:
# Channel interpolation via matrix multiplication
# Let's interpolate channel 2 using channels 1 and 3
interp_channel = 1
InterpMultiplier = np.eye(n_channels)
InterpMultiplier[interp_channel, interp_channel-1] = 1/2
InterpMultiplier[interp_channel, interp_channel+1] = 1/2
InterpMultiplier[interp_channel, interp_channel] = 0
print(InterpMultiplier)
data_interp = InterpMultiplier @ data

# # Plot the interpolated EEG data
# f, axs = plt.subplots(nrows, ncols, figsize=(15, 10))
# axs = axs.flatten()
# for i in range(n_channels):
#     ax = axs.flatten()[i]
#     ax.plot(data_interp[i, :])
#     ax.set_title(f"Channel {i+1}")
#     ax.set_xlabel("Time")
#     ax.set_ylabel("Amplitude")
# plt.show()


[[1.  0.  0.  0.  0. ]
 [0.5 0.  0.5 0.  0. ]
 [0.  0.  1.  0.  0. ]
 [0.  0.  0.  1.  0. ]
 [0.  0.  0.  0.  1. ]]


In [13]:
## Convolution
# Impluse response function (IRF) of a system is the output of the system when the input is an impulse function.
# Convolution is a mathematical operation that takes two functions and produces a third function that represents 
# the amount of overlap between the two functions.
# Convolution is denoted by the symbol *.
# The convolution of two functions f(t) and g(t) is given by:
# (f*g)(t) = integral(f(tau)*g(t-tau)dtau, where tau is the dummy variable of integration.

# Let's define an impulse response function (IRF) of a system, which in case of MRI is the hemodynamic response function (HRF).
# HRF is gamma function that models the response of the blood oxygen level dependent (BOLD) signal to neural activity.
from scipy.stats import gamma
HRF = gamma.pdf(np.linspace(0, 30, 1000), a=6)

# Let's define a neural activity signal that is a square wave signal.
neural_activity = np.zeros(1000)
neural_activity[200:300] = 1

# Convolve the neural activity signal with the HRF
bold_signal = np.convolve(neural_activity, HRF, mode='full')[:1000]

# Plot the neural activity signal and the BOLD signal
f, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].plot(neural_activity)
axs[0].set_title("Neural Activity Signal")
axs[0].set_xlabel("Time")
axs[0].set_ylabel("Amplitude")

axs[1].plot(HRF)
axs[1].set_title("Hemodynamic Response Function (HRF)")
axs[1].set_xlabel("Time")
axs[1].set_ylabel("Amplitude")

axs[2].plot(bold_signal)
axs[2].set_title("BOLD Signal")
axs[2].set_xlabel("Time")
axs[2].set_ylabel("Amplitude")

plt.show()

In [15]:
## Wavelets
# Wavelets are functions that are localized in both time and frequency domains.
# Wavelets are used for time-frequency analysis of signals.
# Mathematically, wavelets are defined as:
# psi(t) = 1/sqrt(a)*psi_0(t-b)/a, where a is the scaling factor, b is the translation factor, and psi_0(t) is the mother wavelet.
# The wavelet transform of a signal f(t) is given by:
# W(a, b) = integral(f(t)*psi(a, b)dt, where psi(a, b) is the wavelet function.
# The wavelet transform is a 2D representation of the signal in the time-frequency domain.

# Let's define a mother wavelet function
def morlet_wavelet(t, f0=10, sigma=0.1):
    return np.exp(2*np.pi*1j*f0*t)*np.exp(-t**2/(2*sigma**2))

# Let's generate a signal that is a sum of two sinusoids
t = np.linspace(0, 1, 1000)
signal = np.sin(2*np.pi*10*t) + np.sin(2*np.pi*20*t)

# Let's compute the wavelet transform of the signal
wavelet_transform = np.zeros((1000, 1000))
for i in range(1000):
    for j in range(1000):
        wavelet_transform[i, j] = np.sum(signal*morlet_wavelet(t-j/1000, f0=10, sigma=0.1))

# Plot the signal and the wavelet transform
f, axs = plt.subplots(1, 2, figsize=(15, 5))
axs[0].plot(t, signal)
axs[0].set_title("Signal")
axs[0].set_xlabel("Time")
axs[0].set_ylabel("Amplitude")  

axs[1].imshow(np.abs(wavelet_transform), aspect='auto', extent=[0, 1, 0, 1000], cmap='jet')
axs[1].set_title("Wavelet Transform")
axs[1].set_xlabel("Time")
axs[1].set_ylabel("Frequency")
plt.show()

/var/folders/0w/p5w49zc93976_p2hxj1zws2w0000gn/T/ipykernel_22511/3877185360.py:22: ComplexWarning: Casting complex values to real discards the imaginary part
  wavelet_transform[i, j] = np.sum(signal*morlet_wavelet(t-j/1000, f0=10, sigma=0.1))


In [2]:
# Path to the EEG file
eegPath = '../../Datasets/EEG/sub-01/eeg/sub-01_task-daf_eeg_filtered.vhdr'

# Load the EEG file using MNE
# MNE has different read formats for different EEG file types
# Here we are using read_raw_edf to read the EEG file
# preload=True loads the data into memory (default is False, which loads the data when needed)
raw = mne.io.read_raw_brainvision(eegPath, preload=True)
elecPos = pd.read_csv('../../Datasets/iEEG/sub-01/eeg/sub-01_electrodes.tsv', sep='\t')
# Add fiducials
fiducials = pd.DataFrame({
    'name': ['Nz', 'LPA', 'RPA'],
    'x': [-4.129838157917329e-18, -0.0729282673627754, 0.08278152042487033],
    'y': [0.10011015398430487, 3.008505424862354e-18, -3.414981080487009e-18],
    'z': [-5.7777898331617076e-33, 3.851859888774472e-34, 3.4666738998970245e-33]
})

# Concatenate the original electrode positions with the fiducials
elecPos = pd.concat([elecPos, fiducials], ignore_index=True)

montage = mne.channels.make_dig_montage(
    ch_pos=dict(zip(elecPos['name'], elecPos[['x', 'y', 'z']].values)),
    coord_frame='head'
)
raw.set_montage(montage)

Extracting parameters from ../../Datasets/EEG/sub-01/eeg/sub-01_task-daf_eeg_filtered.vhdr...
Setting channel info structure...
Reading 0 ... 244237  =      0.000 ...   976.948 secs...


<RawBrainVision | sub-01_task-daf_eeg_filtered.eeg, 66 x 244238 (976.9 s), ~123.1 MB, data loaded>

In [3]:
# Creating a fake channel with a polynomial trend
fake_data = raw.get_data()[0]
t = np.linspace(0, 1, len(fake_data))

np.random.seed(42)
trend = np.random.randn(3)
fake_data += 4e-3 * np.polyval(trend, t)

# Let us look at different ways to remove trend from the data
# First, we can simply mean-center the data
# This is done by subtracting the mean of the data from the data
fake_data_centered = fake_data - np.mean(fake_data)

# Second, we can remove a linear trend from the data
# This can be done using detrend() function of numpy
fake_data_detrended = detrend(fake_data, type='linear')

# Next we can remove a polynomial trend from the data
# This is done by fitting a polynomial model to the data and subtracting the model from the data
# We can use the polyfit() function of numpy to fit a polynomial model to the data
# The polyfit() function returns the coefficients of the polynomial model
# We can use the polyval() function of numpy to evaluate the polynomial model at the data points
# We can then subtract the polynomial model from the data
trend = np.polyfit(t, fake_data, 3)
fake_data_detrended2 = fake_data - np.polyval(trend, t)

# Plot the original data and the detrended data
f, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].plot(t, fake_data)
axs[0, 0].set_title('Original Data')
axs[0, 1].plot(t, fake_data_centered)
axs[0, 1].set_title('Mean-Centered Data')
axs[1, 0].plot(t, fake_data_detrended)
axs[1, 0].set_title('Linear Detrended Data')
axs[1, 1].plot(t, fake_data_detrended2)
axs[1, 1].set_title('Polynomial Detrended Data')
plt.show()